# USA HOUSING

## Importación de las librerias

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print('Setup complete')

Setup complete


## Creación del dataset

In [3]:
df_housing = pd.read_csv('data/USA_Housing.csv')

**ELIMINACIÓN DE VALORES NULOS Y DUPLICADOS**

In [4]:
nulos = df_housing.isna().sum()
nulos

Avg. Area Income                0
Avg. Area House Age             0
Avg. Area Number of Rooms       0
Avg. Area Number of Bedrooms    0
Area Population                 0
Price                           0
Address                         0
dtype: int64

In [5]:
duplicate_sum = df_housing.duplicated().sum()
if duplicate_sum > 0:
  print('Total de duplicados:', duplicate_sum)
else:
  print('No se encontraron duplicados')

No se encontraron duplicados


## Gráficación de los datos